# Boston house prices

The aim of this notebook is to design machine learning methods to predict Boston house prices.  
We will use the data provided by kaggle : [Boston housing data](https://www.kaggle.com/c/boston-housing/data).

## Data

## Models